In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Conv1D, MaxPooling1D, Dropout, Flatten, Lambda
from tensorflow.keras.optimizers import Adam  
from sklearn.model_selection import train_test_split



In [2]:
def load_data_from_folders(base_folder):
    data = []
    labels = []
    ground_truths = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]
    
    for i, ground_truth in enumerate(ground_truths):
        folder_path = os.path.join(base_folder, f'class_{i}')
        for filename in os.listdir(folder_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(folder_path, filename)
                with open(file_path, 'r') as file:
                    
                    numbers = [float(num) for line in file for num in line.split(',')]
                    if len(numbers) == 507:  
                        data.append(numbers)
                        labels.append(ground_truth)
    return np.array(data), np.array(labels)



In [3]:
base_folder = 'B:\jupyter_notebook\localization'
data, labels = load_data_from_folders(base_folder)


In [4]:
index_of_30 = np.where(labels == 30)[0]  
if len(index_of_30) > 0:
    print(f": {data[index_of_30[0]]}")
else:
    print("no result")

: [ 2.990e+01  5.990e+00 -3.000e-02 -2.260e+00 -1.650e+00  2.390e+00
  1.180e+00 -5.200e-01 -1.100e+00  1.020e+00  1.550e+00 -3.300e-01
 -1.190e+00  3.202e+01  7.990e+00 -3.900e-01 -3.130e+00 -1.480e+00
  1.850e+00  1.250e+00 -9.700e-01 -1.350e+00  1.070e+00  9.100e-01
 -3.700e-01 -8.600e-01  3.044e+01  6.590e+00 -5.400e-01 -3.810e+00
  5.000e-02  2.440e+00  1.650e+00 -1.190e+00 -1.070e+00  1.250e+00
  1.600e+00 -5.500e-01 -1.680e+00  3.420e+01  8.390e+00 -2.900e-01
 -2.810e+00 -0.000e+00  2.450e+00  1.040e+00 -1.540e+00 -1.230e+00
  7.900e-01  1.420e+00 -5.000e-02 -1.410e+00  3.309e+01  7.960e+00
 -3.100e-01 -2.750e+00  3.000e-02  1.790e+00  1.170e+00 -1.340e+00
 -8.200e-01  7.000e-01  9.500e-01 -7.800e-01 -1.630e+00  3.285e+01
  7.490e+00  5.800e-01 -2.450e+00 -1.400e-01  3.550e+00  1.450e+00
 -1.140e+00 -1.860e+00  7.500e-01  1.550e+00  4.000e-02 -1.140e+00
  3.351e+01  7.110e+00 -6.400e-01 -3.110e+00 -1.360e+00  2.000e+00
  9.600e-01 -2.700e-01 -1.220e+00 -2.300e-01  5.100e-01 -3.5

In [5]:

train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [6]:

BATCH_SIZE = 32
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).shuffle(buffer_size=len(train_data)).batch(BATCH_SIZE)
validation_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels)).batch(BATCH_SIZE)


input_length = 507  

In [7]:
def cyclic_mse(y_true, y_pred):
    pi = tf.constant(np.pi, dtype=tf.float32)
    deg = tf.constant(180.0, dtype=tf.float32)  
    y_true = tf.cast(y_true, dtype=tf.float32)  
    y_pred = tf.cast(y_pred, dtype=tf.float32)  #y_pred to float32
    return tf.reduce_mean(tf.square(
        tf.atan2(
            tf.sin(y_true * pi / deg - y_pred * pi / deg),
            tf.cos(y_true * pi / deg - y_pred * pi / deg)
        )
    ) * deg / pi)


In [8]:
def cyclic_absolute_difference(y_true, y_pred):
    pi = np.pi
    deg = 180.0
    
    
    y_true_radians = y_true * pi / deg
    y_pred_radians = y_pred * pi / deg
    
   
    angle_diff = np.arctan2(
        np.sin(y_true_radians - y_pred_radians),
        np.cos(y_true_radians - y_pred_radians)
    )
    
   
    angle_diff_deg = np.abs(angle_diff * deg / pi)
    
    return angle_diff_deg.astype(np.float32)

In [9]:
model = Sequential()
model.add(Reshape((int(input_length / 13), 13), input_shape=(input_length, )))
model.add(Conv1D(8, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.25))
model.add(Conv1D(16, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1))  
model.add(tf.keras.layers.Lambda(lambda x: x % 360))  

D:\apps\anaconda\anaconda\envs\812_tf_env\lib\site-packages\keras\src\layers\reshaping\reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
LEARNING_RATE = 0.002
opt = Adam(learning_rate=LEARNING_RATE, beta_1=0.9, beta_2=0.999)
#model.compile(loss=cyclic_mse, optimizer=opt, metrics=[cyclic_mse])
model.compile(loss=cyclic_mse, optimizer=opt)

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_custom_metric = EarlyStopping(
    monitor='val_loss',  
    min_delta=0.001, 
    patience=100,
    restore_best_weights=True
)

In [13]:

EPOCHS = 400
model.fit(train_dataset, epochs=EPOCHS, validation_data=validation_dataset, verbose=2, callbacks=[early_stopping_custom_metric])
#model.fit(train_dataset, epochs=EPOCHS, validation_data=validation_dataset, verbose=2)
# the following printed results are just for testing

Epoch 1/400
18/18 - 1s - 51ms/step - loss: 190.3744 - val_loss: 171.4332
Epoch 2/400
18/18 - 0s - 3ms/step - loss: 187.8223 - val_loss: 168.1603
Epoch 3/400
18/18 - 0s - 3ms/step - loss: 184.7397 - val_loss: 165.7782
Epoch 4/400
18/18 - 0s - 3ms/step - loss: 181.4081 - val_loss: 159.5235
Epoch 5/400
18/18 - 0s - 3ms/step - loss: 177.2576 - val_loss: 150.8441
Epoch 6/400
18/18 - 0s - 3ms/step - loss: 167.7701 - val_loss: 146.1453
Epoch 7/400
18/18 - 0s - 3ms/step - loss: 152.3034 - val_loss: 130.9889
Epoch 8/400
18/18 - 0s - 3ms/step - loss: 131.2046 - val_loss: 109.7428
Epoch 9/400
18/18 - 0s - 3ms/step - loss: 119.5416 - val_loss: 97.3178
Epoch 10/400
18/18 - 0s - 3ms/step - loss: 115.4865 - val_loss: 98.0075
Epoch 11/400
18/18 - 0s - 3ms/step - loss: 114.9221 - val_loss: 87.9752
Epoch 12/400
18/18 - 0s - 3ms/step - loss: 109.5018 - val_loss: 92.4948
Epoch 13/400
18/18 - 0s - 3ms/step - loss: 108.1403 - val_loss: 92.3529
Epoch 14/400
18/18 - 0s - 3ms/step - loss: 109.3851 - val_loss: 

In [14]:

predictions = model.predict(val_data)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [15]:

cyclic_abs_diff_list = []

# 
for i in range(len(predictions)):
    y_pred = predictions[i][0]  
    y_true = val_labels[i]     

    
    if (y_true != 10)and(y_true !=20):
        abs_diff_value = cyclic_absolute_difference(y_true, y_pred)  
        cyclic_abs_diff_list.append(abs_diff_value)
        print(f"Prediction: {y_pred:.2f}, Ground Truth: {y_true}, Absolute Angle Difference: {abs_diff_value:.2f} degrees")


average_abs_diff = np.mean(cyclic_abs_diff_list)
print(f"Average Absolute Angle Difference: {average_abs_diff:.2f} degrees")



Prediction: 34.07, Ground Truth: 30, Absolute Angle Difference: 4.07 degrees
Prediction: 141.89, Ground Truth: 150, Absolute Angle Difference: 8.11 degrees
Prediction: 2.72, Ground Truth: 0, Absolute Angle Difference: 2.72 degrees
Prediction: 303.62, Ground Truth: 300, Absolute Angle Difference: 3.62 degrees
Prediction: 212.49, Ground Truth: 240, Absolute Angle Difference: 27.51 degrees
Prediction: 253.37, Ground Truth: 240, Absolute Angle Difference: 13.37 degrees
Prediction: 41.77, Ground Truth: 30, Absolute Angle Difference: 11.77 degrees
Prediction: 303.27, Ground Truth: 240, Absolute Angle Difference: 63.27 degrees
Prediction: 25.43, Ground Truth: 30, Absolute Angle Difference: 4.57 degrees
Prediction: 346.31, Ground Truth: 330, Absolute Angle Difference: 16.31 degrees
Prediction: 279.22, Ground Truth: 240, Absolute Angle Difference: 39.22 degrees
Prediction: 14.48, Ground Truth: 30, Absolute Angle Difference: 15.52 degrees
Prediction: 303.98, Ground Truth: 330, Absolute Angle Dif

In [16]:

model.save('B:/jupyter_notebook/local_model.h5')


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape (Reshape)                    │ (None, 39, 13)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 39, 8)               │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 20, 8)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 20, 8)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 20, 16)              │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 10, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 10, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 160)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             161 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 1)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,645 (10.34 KB)

 Trainable params: 881 (3.44 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,764 (6.89 KB)

In [18]:
# Fine tune the model:

In [19]:
def load_new_data_from_folders(base_folder):
    data = []
    labels = []
    selected_ground_truths = [0, 90, 180, 270]
    
    for ground_truth in selected_ground_truths:
        folder_path = os.path.join(base_folder, f'class_{ground_truth//30}')
        for filename in os.listdir(folder_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(folder_path, filename)
                with open(file_path, 'r') as file:
                    numbers = [float(num) for line in file for num in line.split(',')]
                    if len(numbers) == 507:  
                        data.append(numbers)
                        labels.append(ground_truth)
    
    return np.array(data), np.array(labels)


base_folder = 'B:/72-91/drone_microphone/NNnetwork/fine_tune/fine_tuned_data'  
new_data, new_labels = load_new_data_from_folders(base_folder)


print(f"Loaded {len(new_data)} samples from the new dataset.")

Loaded 191 samples from the new dataset.


In [20]:
train_data2, val_data2, train_labels2, val_labels2 = train_test_split(new_data, new_labels, test_size=0.2, random_state=42)


In [21]:
train_dataset2 = tf.data.Dataset.from_tensor_slices((train_data2, train_labels2))
train_dataset2 = train_dataset2.shuffle(buffer_size=len(train_data2)).batch(32)

val_dataset2 = tf.data.Dataset.from_tensor_slices((val_data2, val_labels2))
val_dataset2 = val_dataset2.batch(32)

In [22]:
#for layer in model.layers[:-2]:  # forze layers
#   layer.trainable = False
# 
#for layer in model.layers[:-3]:
#    layer.trainable = False
# 
for layer in model.layers:
    layer.trainable = True

# 
#model.layers[-1].trainable = False

In [23]:
LEARNING_RATE2 = 0.003 
opt2 = Adam(learning_rate=LEARNING_RATE2, beta_1=0.9, beta_2=0.999)
model.compile(loss=cyclic_mse, optimizer=opt2)

In [24]:
EPOCHS = 300  
model.fit(train_dataset2, epochs=EPOCHS, validation_data=val_dataset2)

Epoch 1/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 149.7339 - val_loss: 107.2413
Epoch 2/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 155.2622 - val_loss: 105.4173
Epoch 3/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 155.8069 - val_loss: 104.1519
Epoch 4/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 167.7095 - val_loss: 97.7744
Epoch 5/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 151.9868 - val_loss: 94.5704
Epoch 6/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 145.2762 - val_loss: 91.7260
Epoch 7/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 143.6164 - val_loss: 88.7147
Epoch 8/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 146.3915 - val_loss: 88.6096
Epoch 9/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 132.6149 - val_loss: 89.1162
Epoch 10/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 161.8459 - val_loss: 87.9161
Epoch 11/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 151.2604 - val_loss: 86.0924
Epoch 12/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/ste

In [25]:
def load_test_data_from_folders(base_folder):
    data = []
    labels = []
    selected_ground_truths = [0, 30, 60, 90,120,150, 180,210,240,270,300,330]
    
    for ground_truth in selected_ground_truths:
        folder_path = os.path.join(base_folder, f'class_{ground_truth//30}') 
        for filename in os.listdir(folder_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(folder_path, filename)
                with open(file_path, 'r') as file:
                    numbers = [float(num) for line in file for num in line.split(',')]
                    if len(numbers) == 507:  
                        data.append(numbers)
                        labels.append(ground_truth)
    
    return np.array(data), np.array(labels)


In [26]:
test_base_folder = 'B:/72-91/drone_microphone/NNnetwork/fine_tune/test_data'
test_data, test_labels = load_test_data_from_folders(test_base_folder)

In [27]:
predictions2 = model.predict(test_data)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [28]:
cyclic_abs_diff_list2 = []

# 
for i in range(len(predictions2)):
    y_pred = predictions2[i][0]  
    y_true = test_labels[i]     

    
    if y_true != 180:
        abs_diff_value = cyclic_absolute_difference(y_true, y_pred)  
        cyclic_abs_diff_list2.append(abs_diff_value)
        print(f"Prediction: {y_pred:.2f}, Ground Truth: {y_true}, Absolute Angle Difference: {abs_diff_value:.2f} degrees")


average_abs_diff = np.mean(cyclic_abs_diff_list2)
print(f"Average Absolute Angle Difference: {average_abs_diff:.2f} degrees")

Prediction: 19.70, Ground Truth: 0, Absolute Angle Difference: 19.70 degrees
Prediction: 8.94, Ground Truth: 0, Absolute Angle Difference: 8.94 degrees
Prediction: 350.18, Ground Truth: 0, Absolute Angle Difference: 9.82 degrees
Prediction: 349.98, Ground Truth: 0, Absolute Angle Difference: 10.02 degrees
Prediction: 330.35, Ground Truth: 0, Absolute Angle Difference: 29.65 degrees
Prediction: 28.05, Ground Truth: 0, Absolute Angle Difference: 28.05 degrees
Prediction: 351.74, Ground Truth: 0, Absolute Angle Difference: 8.26 degrees
Prediction: 43.25, Ground Truth: 0, Absolute Angle Difference: 43.25 degrees
Prediction: 351.20, Ground Truth: 0, Absolute Angle Difference: 8.80 degrees
Prediction: 53.88, Ground Truth: 0, Absolute Angle Difference: 53.88 degrees
Prediction: 9.71, Ground Truth: 0, Absolute Angle Difference: 9.71 degrees
Prediction: 95.81, Ground Truth: 30, Absolute Angle Difference: 65.81 degrees
Prediction: 71.83, Ground Truth: 30, Absolute Angle Difference: 41.83 degrees

In [ ]:
model.save('B:/jupyter_notebook/localization/fine_tuned_model.h5')